In [0]:
def adls_mount(storage_account_name,container_name):

    #get secrets from Azure key vault
    client_id=dbutils.secrets.get("olympics_tokyo","olympics-tokyo-client-id")
    tenant_id=dbutils.secrets.get("olympics_tokyo","olympics-tokyo-tenant-id")
    client_secrets_key=dbutils.secrets.get("olympics_tokyo","olympics-tokyo-client-secrets-key")

    #configuration
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secrets_key,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",
    extra_configs = configs)
    
    display(dbutils.fs.mounts())


In [0]:
adls_mount('olympicstokyoadlssa','raw')

In [0]:
display(dbutils.fs.ls('/mnt/olympicstokyoadlssa'))

In [0]:
adls_mount('olympicstokyoadlssa','processed')